In [1]:
import numpy as np
import pandas as pd

In [2]:
# Bruce: I was not sure which input values are used for testing
# I assumed the following for now. Will discuss later if we get a chance.
N = 10
q_b = 100
Q = 100
FEE = 0.00363
P_B = 1.5
P_A = 1.25

# Original Code

In [3]:
# Swap exact input - equation (4)
def swap_fwd(x0, f):
    assert(x0 >= 0)
    assert(x0 <= 1)
    x1 = (1 - f) * x0 / (1 + (1 - f) * x0)
    return x1

# Swap exact output - equation (5)
def swap_bwd(x1, f):
    assert(x1 >= 0)
    assert(x1 <= 1)
    x0 = x1/ ( (1 - f) * (1 - x1) )
    return x0

# Optimal arbitrage - equation (15)
def opt_arbitrage(v, f):
    leg0 = (np.sqrt((1 - f) * v) - 1) / (1 - f)
    # Bruce: From what I derived, the term (1-f) should be a division, i.e. leg0 = (np.sqrt((1 - f) * v) - 1) / (1 - f)
    # Originally it was leg0 = (1 - f) * (np.sqrt((1 - f) * v) - 1)
    leg1 = swap_fwd(leg0, f)
    ret = v * leg1 / leg0 - 1
    return leg0, leg1, ret

In [4]:
for n in range(N):
    # Bruce: The fee should not be multiplied by 10?
    # Also maybe we need to update not just q_b, but also other pool reserves?
    q_b = q_b * (1 + 10*FEE)
    
    # Bruce: I guess Q stands for q_a?
    V = q_b * P_B / (Q * P_A)
    # print(V, FEE)
    X_a, X_b, R_est = opt_arbitrage(V, FEE)
    X_0c = swap_bwd(X_a, FEE)
    X_1b  = X_b * q_b / Q             # convert to fraction of q_1c
    X_1c = swap_fwd(X_1b, FEE)
    
    # Bruce: I think this one may have not consider the fact that q_0c could be very different from q_1c?
    R_exact = X_1c / X_0c - 1
    
    print(q_b, V, X_a, X_0c, X_b, X_1b, X_1c, R_est, R_exact)

103.63 1.24356 0.1135365138843609 0.1285446662206531 0.10162779536912489 0.10531688434102411 0.09496904664361151 0.11312437633896066 -0.26119807662347827
107.391769 1.2887012279999999 0.13363257999351408 0.15480664489466864 0.11750235028105006 0.12618785258339613 0.11168736202915662 0.1331474937281376 -0.27853638256194124
111.2900902147 1.3354810825764 0.1540901384024659 0.1828226983197246 0.133096396187518 0.14812309938960333 0.12860516605235844 0.15353079120006496 -0.2965579917902168
115.32992048949362 1.3839590458739235 0.1749156917113814 0.21276970989798172 0.14841488978196482 0.1711667743801096 0.14569741020595825 0.1742807477504693 -0.3152342489172125
119.51639660326224 1.4341967592391467 0.19611585949057792 0.24484915643401606 0.16346270017129969 0.19536472903513197 0.1629386430614596 0.19540395892062734 -0.3345345949543035
123.85484179996065 1.486258101599528 0.21769738038460743 0.2792915368355823 0.17824461042358866 0.2207645802570919 0.18030314681603318 0.21690713889381108 -0

# Redesigned Code

In [5]:
# Number of iterations
N = 10

# Update Assumptions per my understanding
q_0c = 100
q_0a = 50

q_a = 50
q_b = 150

q_1b = 440 # I adjusted it down by 10 from 450 to create the arbitrage opportunity for token C
q_1c = 310 # I adjusted it up by 10 from 300 to create the arbitrage opportunity for token C

# Uniswap currently deloys a 0.3% fee
# https://docs.uniswap.org/contracts/v2/concepts/advanced-topics/fees
# However, every pool and transaction can have a different fee I believe, which we can change later.
# For the time being we assume 0.003 (i.e. 0.3%) for now
f = 0.003

# Assume P_A, P_B do not change from time to time
P_A = q_0c / ((1-f) * q_0a)
P_B = q_1c * (1-f) / q_1b

In [6]:
print("P_A: ", P_A)
print("P_B: ", P_B)

P_A:  2.0060180541624875
P_B:  0.7024318181818182


In [7]:
df = pd.DataFrame(columns=["qb", "V", "Xa", "X0a", "X0c", "Xb", "X1b", "X1c", "R_est", "R_exact"])

for n in range(N):
    
    # Bruce: Now q_b is calculated each time after V is calculated
    V = (q_b * P_B) / (q_a * P_A)
    
    # Bruce: I leave it as it is for now, but was thinking that we also need to update 
    # the quantities for other pool reserves as well for accuracy
    q_b = q_b * (1 + f)
    
    X_a, X_b, R_est = opt_arbitrage(V, f)
    X_0a = X_a * q_a / q_0a
    X_0c = swap_bwd(X_0a, f)
    X_1b  = X_b * q_b / q_1b
    X_1c = swap_fwd(X_1b, f)
    
    # Bruce: I multiplied the numerator and denominator by qc so that only the remaining coin is present
    R_exact = ((X_1c * q_1c) / (X_0c * q_0c)) - 1
    df.loc[n] = [q_b, V, X_a, X_0a, X_0c, X_b, X_1b, X_1c, R_est, R_exact]

In [8]:
df

,qb,V,Xa,X0a,X0c,Xb,X1b,X1c,R_est,R_exact
0,150.450000,1.050487,0.023464,0.023464,0.024101,0.022859,0.007816,0.007733,0.023394,-0.005372
1,150.901350,1.053638,0.025003,0.025003,0.025721,0.024322,0.008341,0.008248,0.024928,-0.005967
2,151.354054,1.056799,0.026544,0.026544,0.027350,0.025782,0.008869,0.008765,0.026464,-0.006568
3,151.808116,1.059970,0.028087,0.028087,0.028986,0.027240,0.009398,0.009283,0.028003,-0.007176
4,152.263541,1.063149,0.029632,0.029632,0.030629,0.028696,0.009930,0.009803,0.029544,-0.007791
5,152.720331,1.066339,0.031180,0.031180,0.032281,0.030150,0.010465,0.010326,0.031087,-0.008412
6,153.178492,1.069538,0.032730,0.032730,0.033940,0.031601,0.011001,0.010849,0.032632,-0.009039
7,153.638028,1.072747,0.034283,0.034283,0.035607,0.033050,0.011540,0.011375,0.034180,-0.009673
8,154.098942,1.075965,0.035838,0.035838,0.037282,0.034498,0.012082,0.011902,0.035730,-0.010314
9,154.561239,1.079193,0.037395,0.037395,0.038964,0.035943,0.012626,0.012431,0.037283,-0.010961
